<a href="https://colab.research.google.com/github/Navya2022/HealthBot/blob/main/Health_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -U transformers datasets
#import transformers
#print(transformers.__version__)
!pip install transformers==4.29.2 tokenizers==0.13.3 datasets==2.14.5 --no-build-isolation --no-cache-dir --upgrade



import transformers
import datasets

print("Transformers version:", transformers.__version__)
print("Datasets version:", datasets.__version__)



Transformers version: 4.29.2
Datasets version: 2.14.5


In [ ]:
import pandas as pd
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Load and clean data
df = pd.read_csv("Diseases_Symptoms.csv")
df = df.dropna(subset=["Symptoms", "Treatments"])

df["input"] = "symptoms = " + df["Symptoms"].str.lower()
df["output"] = "prescription = " + df["Treatments"].str.lower()

# Split into train and validation sets
train_df, eval_df = train_test_split(df[["input", "output"]], test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True))

# Load tokenizer and model
model_name = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained("./HealthBot")
tokenizer = AutoTokenizer.from_pretrained("./HealthBot")

# Preprocessing function
def preprocess_function(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

# Tokenize
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./HealthBot",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

# Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Train
trainer.train()

# Save model
model.save_pretrained("./HealthBot")
tokenizer.save_pretrained("./HealthBot")

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.614218
2,No log,0.612207


KeyboardInterrupt: 

In [ ]:
def get_prescription(symptom_text):
    input_text = "symptoms: " + symptom_text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output_ids = model.generate(
        input_ids,
        max_length=300,
        num_beams=1,
        do_sample=False,
        repetition_penalty=10.0,
        early_stopping=True
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [ ]:
print(get_prescription("nausea, headache,vomiting"))

prescription: antihistamines, dietary changes
